# This notebook is the task that was given as a pre requisite for the project 5 of SRIP 2024
## Pratham Sharda(pratham.sharda@iitgn.ac.in)

# Binary classification with resnet

So here first i have divided the dataset for two classes(I have take it for crab and flamingo) and first have run it without crossvalidation due to time constraint

In [ ]:
import torch
import torchvision
from torchvision import datasets, transforms

traindir = "D:/SRIP/Binary_task/Training"
testdir = "D:/SRIP/Binary_task/Validation"

#transformations
train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                       transforms.ToTensor(),                                
                                       torchvision.transforms.Normalize(
                                           mean=[0.485, 0.456, 0.406],
                                           std=[0.229, 0.224, 0.225],
    ),
                                       ])
test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      torchvision.transforms.Normalize(
                                          mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225],
    ),
                                      ])

#datasets
train_data = datasets.ImageFolder(traindir,transform=train_transforms)
test_data = datasets.ImageFolder(testdir,transform=test_transforms)

#dataloader
trainloader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=16)
testloader = torch.utils.data.DataLoader(test_data, shuffle = True, batch_size=16)

In [ ]:

def make_train_step(model, optimizer, loss_fn):
  def train_step(x,y):
    #make prediction
    yhat = model(x)
    #enter train mode
    model.train()
    #compute loss
    loss = loss_fn(yhat,y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    #optimizer.cleargrads()

    return loss
  return train_step

In [ ]:
from torchvision import datasets, models, transforms
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.resnet18(pretrained=True)

#freeze all params
for params in model.parameters():
  params.requires_grad_ = False

#add a new final layer
nr_filters = model.fc.in_features  #number of input features of last layer
model.fc = nn.Linear(nr_filters, 1)

model = model.to(device)

In [ ]:
from torch.nn.modules.loss import BCEWithLogitsLoss
from torch.optim import lr_scheduler

#loss
loss_fn = BCEWithLogitsLoss() #binary cross entropy with sigmoid, so no need to use sigmoid in the model

#optimizer
optimizer = torch.optim.Adam(model.fc.parameters()) 

#train step
train_step = make_train_step(model, optimizer, loss_fn)

In [ ]:
%%capture
!pip install tqdm
from tqdm import tqdm


losses = []
val_losses = []

epoch_train_losses = []
epoch_test_losses = []

n_epochs = 10
early_stopping_tolerance = 3
early_stopping_threshold = 0.03

for epoch in range(n_epochs):
  epoch_loss = 0
  for i ,data in tqdm(enumerate(trainloader), total = len(trainloader)): #iterate ove batches
    x_batch , y_batch = data
    x_batch = x_batch.to(device) #move to gpu
    y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
    y_batch = y_batch.to(device) #move to gpu


    loss = train_step(x_batch, y_batch)
    epoch_loss += loss/len(trainloader)
    losses.append(loss)
    
  epoch_train_losses.append(epoch_loss)
  print('\nEpoch : {}, train loss : {}'.format(epoch+1,epoch_loss))

  #validation doesnt requires gradient
  with torch.no_grad():
    cum_loss = 0
    for x_batch, y_batch in testloader:
      x_batch = x_batch.to(device)
      y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
      y_batch = y_batch.to(device)

      #model to eval mode
      model.eval()

      yhat = model(x_batch)
      val_loss = loss_fn(yhat,y_batch)
      cum_loss += loss/len(testloader)
      val_losses.append(val_loss.item())


    epoch_test_losses.append(cum_loss)
    print('Epoch : {}, val loss : {}'.format(epoch+1,cum_loss))  
    
    best_loss = min(epoch_test_losses)
    
    #save best model
    if cum_loss <= best_loss:
      best_model_wts = model.state_dict()
    
    #early stopping
    early_stopping_counter = 0
    if cum_loss > best_loss:
      early_stopping_counter +=1

    if (early_stopping_counter == early_stopping_tolerance) or (best_loss <= early_stopping_threshold):
      print("/nTerminating: early stopping")
      break #terminate training
    
#load best model
model.load_state_dict(best_model_wts)

In [ ]:
import matplotlib.pyplot as plt 

def inference(test_data):
  idx = torch.randint(1, len(test_data), (1,))
  sample = torch.unsqueeze(test_data[idx][0], dim=0).to(device)

  if torch.sigmoid(model(sample)) < 0.5:
    print("Prediction : Crab")
  else:
    print("Prediction : Flamingo")


  plt.imshow(test_data[idx][0].permute(1, 2, 0))

inference(test_data)

# Binary classification with cross validation(3 fold) 

In [ ]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn.modules.loss import BCEWithLogitsLoss
from torch.optim import lr_scheduler
import numpy as np
from sklearn.model_selection import KFold
import copy
from tqdm import tqdm
import matplotlib.pyplot as plt

# Define directories (adjust paths as needed)
traindir = "D:/SRIP/Binary_task/Training"  # Assuming this now includes both training and validation data

# Transformations
transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Dataset
dataset = datasets.ImageFolder(traindir, transform=transforms)

# KFold configuration
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

# Training parameters
n_epochs = 10
batch_size = 16

# Loss function
loss_fn = BCEWithLogitsLoss()

# Cross-validation
best_loss_overall = np.inf
best_model_wts_overall = None

for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    print(f"Fold {fold+1}/{kfold.n_splits}")

    # Sampler for splitting data
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Data loaders for training and validation in this fold
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_subsampler)
    testloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=test_subsampler)

    # Model setup
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = models.resnet18(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 1)
    model.to(device)

    # Optimizer
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

    best_loss = np.inf
    best_model_wts = copy.deepcopy(model.state_dict())

    # Training loop
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in tqdm(trainloader, desc=f"Epoch {epoch+1}/{n_epochs}, Fold {fold+1}"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            labels = labels.unsqueeze(1).float()  # Ensure labels are float for BCELoss
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(trainloader.sampler)

        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                labels = labels.unsqueeze(1).float()
                loss = loss_fn(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

        val_loss /= len(testloader.sampler)
        print(f"Fold {fold+1}, Epoch {epoch+1}: Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}")

        if val_loss < best_loss:
            best_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())

    if best_loss < best_loss_overall:
        best_loss_overall = best_loss
        best_model_wts_overall = copy.deepcopy(best_model_wts)

# Load the best model weights from cross-validation
model.load_state_dict(best_model_wts_overall)

# You can now proceed with testing or inference using this model
# For example, here's a simple function to do inference on a single image from the dataset
def inference(dataset, model, device):
    model.eval()
    idx = np.random.randint(0, len(dataset))
    img, _ = dataset[idx]
    img = img.unsqueeze(0).to(device)  # Add batch dimension and transfer to device
    
    with torch.no_grad():
        output = model(img)
        prediction = torch.sigmoid(output).item()

    plt.imshow(dataset[idx][0].permute(1, 2, 0))
    plt.title(f"Prediction: {'Flamingo' if prediction >= 0.5 else 'Crab'}")
    plt.show()

# Example inference
inference(dataset, model, device)


# So now i have created the dataset for 5 class and one vs all classification

So I have take here for one vs rest classification the example of elephant that is is elephant or is not elephant
For 5 class I have converted the dataset into 5 groups where each group is divided based on some particular characteristics(mammals,reptiles,birds,insects and aquatic animals).This has been explicitly mentioned in the notebook below


In [ ]:
# def visualize_conv_layers(model, input_image):
#     # Assuming input_image is a torch.Tensor of shape (C, H, W) and normalized
#     activation = {}
#     def get_activation(name):
#         def hook(model, input, output):
#             activation[name] = output.detach()
#         return hook
    
#     # Register hooks
#     model.conv1.register_forward_hook(get_activation('conv1'))
#     model.conv2.register_forward_hook(get_activation('conv2'))
#     model.conv3.register_forward_hook(get_activation('conv3'))
    
#     # Forward pass
#     output = model(input_image.unsqueeze(0)) # Add batch dimension
    
#     # Plotting
#     for name, act in activation.items():
#         num_feature_maps = act.size(1)
#         # Plot feature maps
#         # You can use matplotlib to create subplots and plot each feature map


Creating one vs rest dataset

In [ ]:
import os
import shutil

# Define the source directory where the current dataset is stored
source_directory = 'D:/SRIP/archive/animals/animals'

# Define the destination directory where the new folder structure will be created
destination_directory = 'D:/SRIP/one_vs_rest_dataset'

# Define the names of the new subdirectories
elephant_dir = os.path.join(destination_directory, 'elephant')
other_animals_dir = os.path.join(destination_directory, 'other_animals')

# Create the destination directory and subdirectories if they don't already exist
os.makedirs(elephant_dir, exist_ok=True)
os.makedirs(other_animals_dir, exist_ok=True)

# Loop through each folder in the source directory
for folder_name in os.listdir(source_directory):
    # Define the path to the current folder
    current_folder_path = os.path.join(source_directory, folder_name)
    
    # Check if the current folder is indeed a directory
    if os.path.isdir(current_folder_path):
        # Determine the destination directory based on whether the folder is 'elephant' or not
        if folder_name.lower() == 'elephant':
            dest_dir = elephant_dir
        else:
            dest_dir = other_animals_dir
        
        # Loop through each file in the current folder
        for filename in os.listdir(current_folder_path):
            # Define the source and destination file paths
            source_file_path = os.path.join(current_folder_path, filename)
            destination_file_path = os.path.join(dest_dir, filename)
            
            # Copy the file from the source to the destination
            shutil.copy(source_file_path, destination_file_path)

print("Dataset reorganization complete.")


3 fold dataset creation for elephant vs rest animals 

In [ ]:
import os
import shutil
from sklearn.model_selection import KFold
import numpy as np

# Define the path to the directory containing the elephant and other_animals folders
dataset_directory = 'D:/SRIP/one_vs_rest_dataset'

# Define the main categories
categories = ['elephant', 'other_animals']

# Initialize KFold with 3 splits
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Process each category separately to split them into folds
for category in categories:
    # Path to the specific category directory
    category_path = os.path.join(dataset_directory, category)
    
    # List all files in the category directory
    files = np.array(os.listdir(category_path))
    
    # Apply KFold splitting
    for fold, (train_idx, val_idx) in enumerate(kf.split(files)):
        # Paths for train and validation directories for this fold
        fold_dir = os.path.join(dataset_directory, f'fold_{fold+1}', category)
        
        # Create the fold directory if it doesn't exist
        os.makedirs(fold_dir, exist_ok=True)
        
        # Validation files for this fold
        val_files = files[val_idx]
        
        # Copy validation files to the fold directory
        for file in val_files:
            src_file_path = os.path.join(category_path, file)
            dst_file_path = os.path.join(fold_dir, file)
            shutil.copy(src_file_path, dst_file_path)

print("3-Fold dataset split complete.")


5 Class dataset creation from the original dataset

In [ ]:
import os
import shutil

# Define your source and destination folders
src_folder = 'D:/SRIP/archive/animals/animals'
dest_folder = 'D:/SRIP/5_class_dataset'

# Mapping of animals to their classes
class_mapping = {
    'Mammals': ["Antelope", "Bear", "Bison", "Cat", "Chimpanzee", "Cow", "Coyote", "Deer", "Dog", "Dolphin", "Elephant", "Fox", "Gorilla", "Kangaroo", "Koala", "Leopard", "Lion", "Otter", "Panda", "Porcupine", "Raccoon", "Reindeer", "Rhinoceros", "Tiger", "Whale", "Wolf", "Zebra"],
    'Birds': ["Bat", "Eagle", "Flamingo", "Hummingbird", "Owl", "Parrot", "Pelecaniformes", "Penguin", "Pigeon", "Sparrow", "Turkey", "Woodpecker"],
    'Aquatic and Amphibious Animals': ["Crab", "Dolphin", "Goldfish", "Jellyfish", "Lobster", "Octopus", "Oyster", "Seahorse", "Seal", "Shark", "Starfish"],
    'Insects and Arthropods': ["Bee", "Beetle", "Butterfly", "Caterpillar", "Cockroach", "Dragonfly", "Fly", "Grasshopper", "Ladybugs", "Mosquito", "Moth"],
    'Reptiles and Others': ["Lizard", "Snake", "Turtle"]  # Assuming 'Crocodile' and 'Tortoise' aren't in your list but could be added if they were.
}

# Create the destination folder if it doesn't exist
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

# Create class folders and move files
for class_name, animals in class_mapping.items():
    class_folder = os.path.join(dest_folder, class_name)
    if not os.path.exists(class_folder):
        os.makedirs(class_folder)
    
    for animal in animals:
        animal_folder = os.path.join(src_folder, animal)
        if os.path.exists(animal_folder):
            for filename in os.listdir(animal_folder):
                src_file = os.path.join(animal_folder, filename)
                dest_file = os.path.join(class_folder, filename)
                # To avoid overwriting files with the same name from different folders, you could add a check here
                shutil.move(src_file, dest_file)

print("Images have been successfully reorganized into class-based folders.")


Craeting 3 fold dataset from the 5 class dataset so that we can have 3 folds for training and testing

In [ ]:
import os
import shutil
from sklearn.model_selection import KFold
import numpy as np

# Define the path to the directory containing the elephant and other_animals folders
dataset_directory = 'D:/SRIP/5_class_dataset'

# Define the main categories
categories = ['Mammals', 'Birds', 'Aquatic and Amphibious Animals', 'Insects and Arthropods', 'Reptiles and Others']

# Initialize KFold with 3 splits
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Process each category separately to split them into folds
for category in categories:
    # Path to the specific category directory
    category_path = os.path.join(dataset_directory, category)
    
    # List all files in the category directory
    files = np.array(os.listdir(category_path))
    
    # Apply KFold splitting
    for fold, (train_idx, val_idx) in enumerate(kf.split(files)):
        # Paths for train and validation directories for this fold
        fold_dir = os.path.join(dataset_directory, f'fold_{fold+1}', category)
        
        # Create the fold directory if it doesn't exist
        os.makedirs(fold_dir, exist_ok=True)
        
        # Validation files for this fold
        val_files = files[val_idx]
        
        # Copy validation files to the fold directory
        for file in val_files:
            src_file_path = os.path.join(category_path, file)
            dst_file_path = os.path.join(fold_dir, file)
            shutil.copy(src_file_path, dst_file_path)

print("3-Fold dataset split complete.")


In [ ]:
# import torch.nn as nn
# import torch.nn.functional as F

# class CustomCNN(nn.Module):
#     def __init__(self, num_classes=2): # Default binary for one-vs-rest
#         super(CustomCNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.fc1 = nn.Linear(128 * 28 * 28, 512) # Adjust the size according to your input
#         self.fc2 = nn.Linear(512, num_classes)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.pool(F.relu(self.conv3(x)))
#         x = x.view(-1, 128 * 28 * 28) # Adjust the size according to your input
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x


# MODEL FOR 5 CLASS 

Now that we have created the datset for the model.


This custom CNN is designed for image classification tasks. It consists of two convolutional layers followed by max pooling layers for feature extraction and spatial downsampling. The convolutional layers use ReLU activation functions. The output from the convolutional layers is flattened and passed through two fully connected layers with ReLU activation and at last having softmax activation function as it is multiclass classification.

In [ ]:
# import torch.nn as nn
# import torch.nn.functional as F

# class CustomCNN(nn.Module):
#     def __init__(self, num_classes=2): # Default binary for one-vs-rest
#         super(CustomCNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.fc1 = nn.Linear(128 * 28 * 28, 512) # Adjust the size according to your input
#         self.fc2 = nn.Linear(512, num_classes)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = self.pool(F.relu(self.conv3(x)))
#         x = x.view(-1, 128 * 28 * 28) # Adjust the size according to your input
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x


In [2]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, ConcatDataset
# from torchvision import datasets, transforms
# from sklearn.metrics import confusion_matrix, classification_report
# import seaborn as sns
# import numpy as np

# class CustomCNN(nn.Module):
#     def __init__(self):
#         super(CustomCNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
#         self.fc1 = nn.Linear(64 * 56 * 56, 1024)
#         self.fc2 = nn.Linear(1024, 5)  # Assuming 5 classes

#     def forward(self, x):
#         x = self.pool(torch.relu(self.conv1(x)))
#         x = self.pool(torch.relu(self.conv2(x)))
#         x = x.view(-1, 64 * 56 * 56)
#         x = torch.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x


Now i am dividing each fold into training and test dataset and then fitting on the model and calculating the metrics to check its performance

In [3]:
# def load_datasets(data_dir, fold):
#     image_transform = transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ])

#     train_datasets = []
#     test_dataset = None
#     for i in range(1, 4):  # For fold 1, fold 2, fold 3
#         fold_path = f'{data_dir}/fold_{i}'
#         if i == fold:
#             test_dataset = datasets.ImageFolder(root=fold_path, transform=image_transform)
#         else:
#             train_datasets.append(datasets.ImageFolder(root=fold_path, transform=image_transform))

#     train_dataset = ConcatDataset(train_datasets)
#     return train_dataset, test_dataset


In [4]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = CustomCNN().to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# data_dir = 'D:/folder/SRIP_Task/5_class_dataset'

# for fold in range(1, 4):
#     print(f"Training on Fold {fold}")
#     train_dataset, test_dataset = load_datasets(data_dir, fold)
#     train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#     test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
#     # Training
#     model.train()
#     for epoch in range(10):  # Number of epochs
        
#         for images, labels in train_loader:
#             images, labels = images.to(device), labels.to(device)
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             print(f"Epoch {epoch+1}, Loss: {loss.item()}")
            
    
#     # Evaluation
#     model.eval()
#     correct = 0
#     total = 0
#     true_labels = []
#     predicted_labels = []
#     with torch.no_grad():
#         for images, labels in test_loader:
#             images, labels = images.to(device), labels.to(device)
#             outputs = model(images)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
#             true_labels.extend(labels.cpu().numpy())
#             predicted_labels.extend(predicted.cpu().numpy())
#     print(f'Accuracy of the network on fold {fold} test images: {100 * correct / total}%')
    
#     # Compute confusion matrix
#     cm = confusion_matrix(true_labels, predicted_labels)
#     sns.heatmap(cm, annot=True, fmt='d')
#     plt.title(f'Confusion Matrix for Fold {fold}')
#     plt.ylabel('Actual Label')
#     plt.xlabel('Predicted Label')
#     plt.show()
    



Training on Fold 1


KeyboardInterrupt: 

In [1]:
import os
import random
import shutil

# Set the path to the folder containing the images
folder_path = "D:/folder/SRIP_Task/5_class_dataset/fold_1"

# Define the ratio of images to be used for training (0.8 = 80% training, 0.2 = 20% testing)
train_ratio = 0.8

# Create train and test directories if they don't exist
train_dir = os.path.join(folder_path, "train")
test_dir = os.path.join(folder_path, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# List all folders in the main folder
subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

for subfolder in subfolders:
    # Get the name of the subfolder (elephant or not elephant)
    label = os.path.basename(subfolder)
    
    # List all images in the subfolder
    images = [f for f in os.listdir(subfolder) if f.endswith(".jpg")]
    
    # Shuffle the images
    random.shuffle(images)
    
    # Calculate the number of images for training
    num_train = int(train_ratio * len(images))
    
    # Split the images into train and test sets
    train_images = images[:num_train]
    test_images = images[num_train:]
    
    # Copy train images to the train directory
    for image in train_images:
        src = os.path.join(subfolder, image)
        dst = os.path.join(train_dir, label, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy(src, dst)
    
    # Copy test images to the test directory
    for image in test_images:
        src = os.path.join(subfolder, image)
        dst = os.path.join(test_dir, label, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy(src, dst)

print("Dataset split completed successfully.")


Dataset split completed successfully.


In [2]:
import os
import random
import shutil

# Set the path to the folder containing the images
folder_path = "D:/folder/SRIP_Task/5_class_dataset/fold_2"

# Define the ratio of images to be used for training (0.8 = 80% training, 0.2 = 20% testing)
train_ratio = 0.8

# Create train and test directories if they don't exist
train_dir = os.path.join(folder_path, "train")
test_dir = os.path.join(folder_path, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# List all folders in the main folder
subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

for subfolder in subfolders:
    # Get the name of the subfolder (elephant or not elephant)
    label = os.path.basename(subfolder)
    
    # List all images in the subfolder
    images = [f for f in os.listdir(subfolder) if f.endswith(".jpg")]
    
    # Shuffle the images
    random.shuffle(images)
    
    # Calculate the number of images for training
    num_train = int(train_ratio * len(images))
    
    # Split the images into train and test sets
    train_images = images[:num_train]
    test_images = images[num_train:]
    
    # Copy train images to the train directory
    for image in train_images:
        src = os.path.join(subfolder, image)
        dst = os.path.join(train_dir, label, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy(src, dst)
    
    # Copy test images to the test directory
    for image in test_images:
        src = os.path.join(subfolder, image)
        dst = os.path.join(test_dir, label, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy(src, dst)

print("Dataset split completed successfully.")


Dataset split completed successfully.


In [3]:
import os
import random
import shutil

# Set the path to the folder containing the images
folder_path = "D:/folder/SRIP_Task/5_class_dataset/fold_3"

# Define the ratio of images to be used for training (0.8 = 80% training, 0.2 = 20% testing)
train_ratio = 0.8

# Create train and test directories if they don't exist
train_dir = os.path.join(folder_path, "train")
test_dir = os.path.join(folder_path, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# List all folders in the main folder
subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

for subfolder in subfolders:
    # Get the name of the subfolder (elephant or not elephant)
    label = os.path.basename(subfolder)
    
    # List all images in the subfolder
    images = [f for f in os.listdir(subfolder) if f.endswith(".jpg")]
    
    # Shuffle the images
    random.shuffle(images)
    
    # Calculate the number of images for training
    num_train = int(train_ratio * len(images))
    
    # Split the images into train and test sets
    train_images = images[:num_train]
    test_images = images[num_train:]
    
    # Copy train images to the train directory
    for image in train_images:
        src = os.path.join(subfolder, image)
        dst = os.path.join(train_dir, label, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy(src, dst)
    
    # Copy test images to the test directory
    for image in test_images:
        src = os.path.join(subfolder, image)
        dst = os.path.join(test_dir, label, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy(src, dst)

print("Dataset split completed successfully.")


Dataset split completed successfully.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(64 * 56 * 56, 1024)
        self.fc2 = nn.Linear(1024, 5)  # Assuming 5 classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 56 * 56)  # Flatten the output for the fully connected layer
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [3]:
def load_datasets(fold_path):
    image_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    train_dir = os.path.join(fold_path, 'train')
    test_dir = os.path.join(fold_path, 'test')

    train_dataset = datasets.ImageFolder(root=train_dir, transform=image_transform)
    test_dataset = datasets.ImageFolder(root=test_dir, transform=image_transform)

    return train_dataset, test_dataset


In [4]:
def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, device, epochs=15):
    model.train()
    for epoch in range(epochs):
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            print(f"Epoch {epoch+1}, Loss: {loss.item()}")
        print(f"Epoch {epoch+1} completed.")
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy}%')


In [5]:
parent_dir = 'D:/folder/SRIP_Task/5_class_dataset'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CustomCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for fold in range(1, 4):  # Assuming there are 3 folds
    print(f"Training on Fold {fold}")
    fold_path = f'{parent_dir}/fold_{fold}'
    train_dataset, test_dataset = load_datasets(fold_path)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, device, epochs=15)


Training on Fold 1


KeyboardInterrupt: 

# one vs rest classification

In [1]:
import os
import random
import shutil

# Set the path to the folder containing the images
folder_path = "D:/folder/SRIP_Task/one_vs_rest_dataset/fold_1"

# Define the ratio of images to be used for training (0.8 = 80% training, 0.2 = 20% testing)
train_ratio = 0.8

# Create train and test directories if they don't exist
train_dir = os.path.join(folder_path, "train")
test_dir = os.path.join(folder_path, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# List all folders in the main folder
subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

for subfolder in subfolders:
    # Get the name of the subfolder (elephant or not elephant)
    label = os.path.basename(subfolder)
    
    # List all images in the subfolder
    images = [f for f in os.listdir(subfolder) if f.endswith(".jpg")]
    
    # Shuffle the images
    random.shuffle(images)
    
    # Calculate the number of images for training
    num_train = int(train_ratio * len(images))
    
    # Split the images into train and test sets
    train_images = images[:num_train]
    test_images = images[num_train:]
    
    # Copy train images to the train directory
    for image in train_images:
        src = os.path.join(subfolder, image)
        dst = os.path.join(train_dir, label, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy(src, dst)
    
    # Copy test images to the test directory
    for image in test_images:
        src = os.path.join(subfolder, image)
        dst = os.path.join(test_dir, label, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy(src, dst)

print("Dataset split completed successfully.")


Dataset split completed successfully.


In [2]:
import os
import random
import shutil

# Set the path to the folder containing the images
folder_path = "D:/folder/SRIP_Task/one_vs_rest_dataset/fold_2"

# Define the ratio of images to be used for training (0.8 = 80% training, 0.2 = 20% testing)
train_ratio = 0.8

# Create train and test directories if they don't exist
train_dir = os.path.join(folder_path, "train")
test_dir = os.path.join(folder_path, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# List all folders in the main folder
subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

for subfolder in subfolders:
    # Get the name of the subfolder (elephant or not elephant)
    label = os.path.basename(subfolder)
    
    # List all images in the subfolder
    images = [f for f in os.listdir(subfolder) if f.endswith(".jpg")]
    
    # Shuffle the images
    random.shuffle(images)
    
    # Calculate the number of images for training
    num_train = int(train_ratio * len(images))
    
    # Split the images into train and test sets
    train_images = images[:num_train]
    test_images = images[num_train:]
    
    # Copy train images to the train directory
    for image in train_images:
        src = os.path.join(subfolder, image)
        dst = os.path.join(train_dir, label, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy(src, dst)
    
    # Copy test images to the test directory
    for image in test_images:
        src = os.path.join(subfolder, image)
        dst = os.path.join(test_dir, label, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy(src, dst)

print("Dataset split completed successfully.")


Dataset split completed successfully.


In [3]:
import os
import random
import shutil

# Set the path to the folder containing the images
folder_path = "D:/folder/SRIP_Task/one_vs_rest_dataset/fold_3"

# Define the ratio of images to be used for training (0.8 = 80% training, 0.2 = 20% testing)
train_ratio = 0.8

# Create train and test directories if they don't exist
train_dir = os.path.join(folder_path, "train")
test_dir = os.path.join(folder_path, "test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# List all folders in the main folder
subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

for subfolder in subfolders:
    # Get the name of the subfolder (elephant or not elephant)
    label = os.path.basename(subfolder)
    
    # List all images in the subfolder
    images = [f for f in os.listdir(subfolder) if f.endswith(".jpg")]
    
    # Shuffle the images
    random.shuffle(images)
    
    # Calculate the number of images for training
    num_train = int(train_ratio * len(images))
    
    # Split the images into train and test sets
    train_images = images[:num_train]
    test_images = images[num_train:]
    
    # Copy train images to the train directory
    for image in train_images:
        src = os.path.join(subfolder, image)
        dst = os.path.join(train_dir, label, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy(src, dst)
    
    # Copy test images to the test directory
    for image in test_images:
        src = os.path.join(subfolder, image)
        dst = os.path.join(test_dir, label, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copy(src, dst)

print("Dataset split completed successfully.")


Dataset split completed successfully.


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # Assuming 3-channel input images
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)  # Adjust size according to your image size
        self.fc2 = nn.Linear(512, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 8 * 8)  # Adjust flattening size according to your image size
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [15]:
from sklearn.metrics import classification_report, confusion_matrix
def get_data_loader(root_dir, fold, batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    # Adjust the path to where your data is stored
    train_data = ImageFolder(root=os.path.join(root_dir, f'fold_{fold}', 'train'), transform=transform)
    test_data = ImageFolder(root=os.path.join(root_dir, f'fold_{fold}', 'test'), transform=transform)
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
    return train_loader, test_loader


In [16]:
def train_and_evaluate(model, train_loader, test_loader, epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            print(f"Epoch {epoch+1}, Loss: {loss.item()}")
        
        print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')
    
    # Evaluation
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy}%')

    print('Classification Report:')
    print(classification_report(all_labels, all_preds, target_names=['Not Elephant', 'Elephant']))
    
    print('Confusion Matrix:')
    cm = confusion_matrix(all_labels, all_preds)
    print(cm)
    return accuracy


In [17]:
root_dir = 'D:/folder/SRIP_Task/one_vs_rest_dataset'  # Adjust this path
n_folds = 3
batch_size = 32
epochs = 10

accuracies = []

for fold in range(1, n_folds+1):
    print(f'Starting fold {fold}')
    train_loader, test_loader = get_data_loader(root_dir, fold, batch_size)
    model = SimpleCNN()
    accuracy = train_and_evaluate(model, train_loader, test_loader, epochs)
    accuracies.append(accuracy)

# Calculate and print the average accuracy
average_accuracy = np.mean(accuracies)
print(f'Average Accuracy: {average_accuracy}%')


Starting fold 1
Epoch 1, Loss: 0.6961296200752258
Epoch 1, Loss: 0.327700138092041
Epoch 1, Loss: 0.04799414426088333
Epoch 1, Loss: 0.0015845266170799732
Epoch 1, Loss: 8.861989044817165e-05
Epoch 1, Loss: 0.5008435845375061
Epoch 1, Loss: 2.045147539320169e-06
Epoch 1, Loss: 1.7806787582230754e-06
Epoch 1, Loss: 3.1701736133982195e-06
Epoch 1, Loss: 3.0397823138628155e-06
Epoch 1, Loss: 2.2277015432337066e-06
Epoch 1, Loss: 7.081425792421214e-06
Epoch 1, Loss: 2.5108149657171452e-06
Epoch 1, Loss: 2.3208322090795264e-06
Epoch 1, Loss: 0.4275161027908325
Epoch 1, Loss: 0.3947215676307678
Epoch 1, Loss: 0.4502102732658386
Epoch 1, Loss: 0.000518253946211189
Epoch 1, Loss: 0.0022971753496676683
Epoch 1, Loss: 0.005846443120390177
Epoch 1, Loss: 0.013880755752325058
Epoch 1, Loss: 0.024269351735711098
Epoch 1, Loss: 0.03464173525571823
Epoch 1, Loss: 0.035341206938028336
Epoch 1, Loss: 0.13371872901916504
Epoch 1, Loss: 0.1361151784658432
Epoch 1, Loss: 0.03951061889529228
Epoch 1, Loss:

c:\Users\Pratham Sharda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Pratham Sharda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Pratham Sharda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

              precision    recall  f1-score   support

Not Elephant       0.00      0.00      0.00         4
    Elephant       0.99      1.00      0.99       356

    accuracy                           0.99       360
   macro avg       0.49      0.50      0.50       360
weighted avg       0.98      0.99      0.98       360

Confusion Matrix:
[[  0   4]
 [  0 356]]
Starting fold 2
Epoch 1, Loss: 0.709985613822937
Epoch 1, Loss: 0.22198162972927094
Epoch 1, Loss: 0.01612135022878647
Epoch 1, Loss: 0.0008069262257777154
Epoch 1, Loss: 2.355145988985896e-05
Epoch 1, Loss: 3.464517135398637e-07
Epoch 1, Loss: 1.9278863668441772
Epoch 1, Loss: 0.5026818513870239
Epoch 1, Loss: 5.10364031924837e-07
Epoch 1, Loss: 0.37063661217689514
Epoch 1, Loss: 0.00013551549636758864
Epoch 1, Loss: 0.0006127459346316755
Epoch 1, Loss: 0.0018440569983795285
Epoch 1, Loss: 0.18586048483848572
Epoch 1, Loss: 0.013355427421629429
Epoch 1, Loss: 0.15267829596996307
Epoch 1, Loss: 0.12276867777109146
Epoch 1, L

c:\Users\Pratham Sharda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Pratham Sharda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Pratham Sharda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

Epoch 1, Loss: 0.6932142972946167
Epoch 1, Loss: 0.2723560333251953
Epoch 1, Loss: 0.04337079077959061
Epoch 1, Loss: 0.20579810440540314
Epoch 1, Loss: 0.0005400777445174754
Epoch 1, Loss: 0.00010845762881217524
Epoch 1, Loss: 1.9717092072824016e-05
Epoch 1, Loss: 7.02209126757225e-06
Epoch 1, Loss: 0.4764867126941681
Epoch 1, Loss: 4.540860118140699e-06
Epoch 1, Loss: 6.99962220096495e-06
Epoch 1, Loss: 0.7275425791740417
Epoch 1, Loss: 8.845605771057308e-05
Epoch 1, Loss: 0.3282538056373596
Epoch 1, Loss: 0.20424197614192963
Epoch 1, Loss: 0.003546477295458317
Epoch 1, Loss: 0.009048271924257278
Epoch 1, Loss: 0.021714622154831886
Epoch 1, Loss: 0.028473839163780212
Epoch 1, Loss: 0.1313093602657318
Epoch 1, Loss: 0.14086773991584778
Epoch 1, Loss: 0.05083353444933891
Epoch 1, Loss: 0.13571566343307495
Epoch 1, Loss: 0.04633050411939621
Epoch 1, Loss: 0.04050522670149803
Epoch 1, Loss: 0.029756663367152214
Epoch 1, Loss: 0.02161243185400963
Epoch 1, Loss: 0.1365460604429245
Epoch 1,

c:\Users\Pratham Sharda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Pratham Sharda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Pratham Sharda\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh